# What is wrong with Continual Learning in Medical Image Segmentation - Visualization

## of AE reconstructions
By using these commands you can generate reconstructions images using your trained autoencoders.
The images will be stored to .../evaluation/nnUNet_ext/2d/Task008_mHeartA/Task008_mHeartA/<TrainerMethod\>__nnUNetPlansv2.1/Generic_UNet/SEQ/head_None/fold_0/Preds_Task008_mHeartA


In [ ]:
!nnUNet_inference 2d nnUNetTrainerExpertGateUNet -f 0 -trained_on 8 -use_model 8 -d 0 -evaluate_on 8
!nnUNet_inference 2d nnUNetTrainerExpertGateUNet -f 0 -trained_on 9 -use_model 9 -d 0 -evaluate_on 8

!nnUNet_inference 2d nnUNetTrainerExpertGateMonai -f 0 -trained_on 8 -use_model 8 -d 0 -evaluate_on 8
!nnUNet_inference 2d nnUNetTrainerExpertGateMonai -f 0 -trained_on 9 -use_model 9 -d 0 -evaluate_on 8

You can visualize these files by using some medical imaging software (e.g. [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK) )).

Since the nnUNet framework preprocesses the images before passing it to the network, getting the original image with the same preprocessing applied as the reconstructions is a bit tricky: 
The way we did this, is removing the reconstructed images from one of our autoencoders (if inference has been done before).
After that, we change the forward method of that autoencoder to be an identity mapping. You can find the classes at .../Lifelong-nnUNet/nnunet_ext/network_architecture. The classes you may want to adapt are:
* expert_gate_UNet
* expert_gate_autoencoder
* ExpertGateMonaiAutoencoder (in expert_gate_monai_ae.py)<br>

Note, that you only need to change a single autoencoder. After changing, the forward method could look like this:

In [ ]:
    def forward(self, x):
        return x
        skips = []
        seg_outputs = []
        for d in range(len(self.conv_blocks_context) - 1):#down
            x = self.conv_blocks_context[d](x)
            skips.append(x)
            if not self.convolutional_pooling:
                x = self.td[d](x)

        x = self.conv_blocks_context[-1](x)

        for u in range(len(self.tu)):#up
            x = self.tu[u](x)
            x = torch.cat((x, skips[-(u + 1)]), dim=1)
            x = self.conv_blocks_localization[u](x)
            seg_outputs.append(self.seg_outputs[u](x))

        return seg_outputs[-1]

After applying inference with that model, the original images will be exported with the same preprocessing applied. **Do not forget to revert the changes after you are done.**
## of residual images
After exporting your images as well as the original image, you can compute the residual images. Our exported images of the reconstructions using the modesl trained on task 8 and 9 are called _8.png_ , _9.png_ respectily. The original image is called _gt.png_. To get good visual results you may need to adjust the precentage parameter.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

FOLDER = "Siemens_UNet/"
image1 = "8.png"
image2 = "9.png"
percentage = 0.6 #should be in range (0,1]. 1: Use whole image value range.


# Load the two images
gt = plt.imread(FOLDER + "gt.png")
img1 = plt.imread(FOLDER + image1)
img2 = plt.imread(FOLDER + image2)

# Compute the residual image
residual_image1 = np.abs(gt - img1)
residual_image2 = np.abs(gt - img2)

residual_image1 = residual_image1[:,:,0]
residual_image2 = residual_image2[:,:,0]

maX = max(np.amax(residual_image1), np.amax(residual_image2))
miN = min(np.amin(residual_image1), np.amax(residual_image2))
maX -= percentage * maX

print(np.mean(np.square(residual_image1)))
print(np.mean(np.square(residual_image2)))




N = 256
A = 256 //4
vals = np.ones((N, 4))

vals[:, 0] = np.concatenate((np.linspace(  0/256,    0/256, A), np.linspace(  0/256,    0/256, A), np.linspace(    0/256,  256/256, A), np.linspace(256/256,   256/256, N - 3*A)))
vals[:, 1] = np.concatenate((np.linspace(  0/256,  256/256, A), np.linspace(256/256,  256/256, A), np.linspace(  256/256,  256/256, A), np.linspace(256/256,  0/256, N - 3*A)))
vals[:, 2] = np.concatenate((np.linspace(256/256,  256/256, A), np.linspace(256/256,    0/256, A), np.linspace(    0/256,    0/256, A), np.linspace(  0/256,  0/256, N - 3*A)))
newcmp = matplotlib.colors.ListedColormap(vals)



# Show the residual image
plt.imshow(residual_image1, vmin=miN, vmax=maX, cmap=newcmp)
plt.colorbar()
plt.savefig(FOLDER + "residual_" + image1 + ".svg")

plt.imshow(residual_image2, vmin=miN, vmax=maX, cmap=newcmp)
plt.savefig(FOLDER + "residual_" + image2 + ".svg")